In [219]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

NASA Mars News

In [193]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/bradmarchant/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [194]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [195]:
browser.visit(url)

In [196]:
html = browser.html

In [197]:
soup = bs(html, 'html.parser')
# print(soup.prettify())

In [198]:
headlines = soup.find('li', class_="slide")
headlines

<li class="slide"><div class="image_and_description_container"><a href="/news/8756/my-culture-my-voice/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="Christina Hernandez inside the mission control area at JPL. " src="/system/news_items/list_view_images/8756_25254_Christina_hernandez_mainmax-320x240.png"/></div><div class="bottom_gradient"><div><h3>My Culture, My Voice</h3></div></div></a><div class="list_text"><div class="list_date">September 16, 2020</div><div class="content_title"><a href="/news/8756/my-culture-my-voice/" target="_self">My Culture, My Voice</a></div><div class="article_teaser_body">In honor of Hispanic Heritage Month, Christina Hernandez, an instru

In [199]:
title = headlines.find('div', class_="content_title").a.text.strip()
title

'My Culture, My Voice'

In [200]:
news_p = headlines.find('div', class_="article_teaser_body").text

In [201]:
news_p

'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.'

In [202]:
image_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

In [203]:
browser.click_link_by_partial_text('FULL IMAGE')

/Users/bradmarchant/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [204]:
browser.click_link_by_partial_text('more info')

In [205]:
html2 = browser.html
soup = bs(html2, 'html.parser')

In [206]:
url= soup.find('figure',class_='lede').a['href']

In [207]:
url

'/spaceimages/images/largesize/PIA18429_hires.jpg'

In [208]:
featured_image_url = "https://www.jpl.nasa.gov"+url

In [209]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18429_hires.jpg'

In [210]:
mars_url="https://space-facts.com/mars/"
browser.visit(mars_url)

In [211]:
df=pd.read_html(mars_url)[0]

In [212]:
mars_data=df.to_html()

In [213]:
hemisphere_titles =[]

In [214]:
hem_url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hem_url)

In [215]:
header = browser.find_by_css(".description")
header

In [216]:
hemisphere_image_urls =[]
for i in range(len(header)):
    hemisphere={}
    description = browser.find_by_css(".description")
    add=description[i].find_by_tag("h3").text
    browser.links.find_by_partial_text(add).click()
    hemisphere['title']=add
    sample=browser.find_by_text('Sample').first['href']
    hemisphere['img_url']=sample
    hemisphere_image_urls.append(hemisphere)
    
    browser.back()
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [221]:
MarsDB={
'newsTitle':title,
'newsP':news_p,
'featureImage':featured_image_url,
'dataTable':mars_data,
'hemispheres':hemisphere_image_urls
}

{'newsTitle': 'My Culture, My Voice',
 'newsP': 'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.',
 'featureImage': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18429_hires.jpg',
 'dataTable': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </t